In [ ]:
import torch

In [ ]:
model_file = '/home/commaai-03/Public/ye/test/DFA_model_latest.pkl'

In [ ]:
model = torch.load(model_file)

In [ ]:
def toTensor(data):
    '''Change the data range to [0.0, 1.0]
    Return [B, C, H, W]
    '''
    assert data.ndim == 2 or data.ndim == 3
    if data.ndim == 2:
        # depth
        data = data[..., np.newaxis]
        mask = data > 10000
        data[mask] = 0
        data = data / 10000.
    elif data.ndim == 3:
        # image
        data = data / 255.
    data = np.transpose(data, (2, 0, 1))
    data = data[np.newaxis, ...]
    return data

def normalize(data, mean, std):
    assert data.ndim == 4
    assert data.shape[1] == len(mean) == len(std)
    batch_size = data.shape[0]
    mean, std = np.asarray(mean), np.asarray(std)
    mean = mean[..., np.newaxis, np.newaxis]
    std = std[..., np.newaxis, np.newaxis]
    for i in range(batch_size):
        data[i, ...] = (data[i, ...] - mean) / std
    return data

In [ ]:
img = '/home/commaai-03/Public/ye/train_jpg/rs07_1564022349_55_flip2.jpg'
depth = '/home/commaai-03/Public/ye/train_jpg/rs07_1564022349_55_flip2.pgm'

import cv2
import numpy as np

img = cv2.imread(img)
depth = cv2.imread(depth, 2)

In [ ]:
data = toTensor(depth)

In [ ]:
data = normalize(data, [0.1864497], [0.07711394])

In [ ]:
img_data_ = toTensor(img)

In [ ]:
img_data = normalize(img_data_, [0.368, 0.393, 0.404], [0.286, 0.290, 0.296])

In [ ]:
a = np.concatenate((img_data, data), axis=1)

In [ ]:
def topk(arr, K, mode='max'):
    '''Same as Pytorch torch.topk function
    Return the index of the max or min K-th values.
    
    Args:
    ------
    arr: ndarray, Input array.
    K: the K-th.
    mode: max or min.
    '''
    if not isinstance(K, int):
        raise TypeError('K must be a int.')
    if not isinstance(mode, str):
        raise TypeError('mode must be a string.')
    mode = mode.lower()
    if mode == 'max':
        K = -K
    elif mode == 'min':
        K = K
    else:
        raise Exception('mode must be in max or min.')
    
    idx = np.argpartition(arr, K)[K:]
    topk_idx = idx[np.argsort(arr[idx])][::-1]
    return topk_idx


def view_as_block(arr_in, block_shape):
    '''Block view of the input n-dimensional array.
    
    Blocks are non-overlapping views of the input array.
    Args:
    ------
    arr_in: ndarray, N-d input array.
    block_shape: tuple, The shape of the blcok or the kernel size.
    '''
    
    if not isinstance(block_shape, tuple):
        raise TypeError('block_shape must be a tuple.')
        
    block_shape = np.array(block_shape)
    if block_shape.size != arr_in.ndim:
        raise ValueError('block_sahep must have the same length as arr_in.shape.')

In [ ]:
def _gather_feat(feature, idx):
    dim = feature.shape[2]
    idx = np.expand_dims(idx, 2)
    idx = np.broadcast_to(idx, (idx.shape[0], idx.shape[1], dim))
    

def _topk(heatmap, K=10):
    B, C, H, W = heatmap.shape
    # Numpy topk
    idxs = np.argpartition(heatmap.reshape(B, C, -1), -K)[-K:]
    topk_idxs = idxs[np.argsort(heatmap[idxs])][::-1]
    topk_scores = heatmap[topk_idxs]
    topk_idxs %= (H * W)
    topk_ys = np.round(topk_idxs / W)
    topk_xs = np.round(topk_idxs % W)
    
    idx = np.argpartition(topk_scores.reshape(B, -1), -K)[-K:]
    topk_idx = idx[np.argsort(topk_scores[idx])][::-1]
    topk_score = topk_scores[topk_idx]
    topk_clses = np.int(np.round(topk_idx / K))
    
    
    
    

def ctdet_decode(heatmap, wh, offset, threshold=0.5):
    
    B, C, H, W = heatmap.shape
    
